###### Aula 06 - Perceptron

In [57]:
import numpy as np
from sklearn import datasets
from sklearn.metrics import accuracy_score

### Iris DataSet

In [29]:
iris = datasets.load_iris()
print("dir(iris) =", dir(iris))
print("iris.feature_names =", iris.feature_names)
print("iris.target_names =", iris.target_names)
print("iris.target =", iris.target)
print("iris.data =", iris.data)

dir(iris) = ['DESCR', 'data', 'feature_names', 'target', 'target_names']
iris.feature_names = ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
iris.target_names = ['setosa' 'versicolor' 'virginica']
iris.target = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
iris.data = [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2

In [35]:
def randomize(X):
    np.random.seed(7483)
    np.random.shuffle(X)

In [36]:
randomize(iris.data)
randomize(iris.target)

### Classe do Perceptron

O perceptron é uma unidade básica de aprendizado por máquina, que possui várias entradas ligadas diretamente a uma saída.

In [52]:
class Perceptron:
    
    def __init__(self):
        return
    def fit(self, X, Y):
        return
    def predict(self, X):
        return

### Particionamento
Irei particionar os dados em N partes, sendo que:
- 1/N será de teste
- (N-1)/N será de aprendizado

In [59]:
N = 3
for i in range(0, N):
    p = Perceptron()
    for j in range(0, N):
        if (i != j):
            p.fit(X[j::N], Y[j::N])
    Y_pred = p.predict(X[i::N])
    acc = accuracy_score(Y[i::N], Y_pred)
